In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
data['rsi_14'] = data['rsi_14'].shift(1)

from talib import MA, SMA, EMA, WMA
data['ma_9'] = MA(data['close'], timeperiod=9)
data['ma_9'] = data['ma_9'].shift(1)
data['sma_9'] = SMA(data['close'], timeperiod=9)
data['sma_9'] = data['sma_9'].shift(1)
data['wma_9'] = WMA(data['close'], timeperiod=9)
data['wma_9'] = data['wma_9'].shift(1)

from talib import MACD
data['macd'], data['signal'], data['hist'] = MACD(data['close'])
data['macd'] = data['macd'].shift(1)
data['signal'] = data['signal'].shift(1)
data['hist'] = data['hist'].shift(1)

from talib import ADX
data['adx'] = ADX(data['high'], data['low'], data['close'])
data['adx'] = data['adx'].shift(1)

from talib import ATR
data['atr'] = ATR(high=data['high'], low=data['low'], close=data['close'], timeperiod=14)
data['atr'] = data['atr'].shift(1)

from talib import SAR
data['sar'] = SAR(high=data['high'], low=data['low'], acceleration=0.02, maximum=0.2)
data['sar'] = data['sar'].shift(1)

from talib import TEMA
data['tema'] = TEMA(data['close'], timeperiod=14)
data['tema'] = data['tema'].shift(1)

from talib import ROC
data['roc'] = ROC(data['close'], timeperiod=14)
data['roc'] = data['roc'].shift(1)

data.dropna(axis=0, inplace=True)

In [2]:
data.shape

(16222, 18)

In [3]:
data.columns

Index(['timestamp', 'date', 'open', 'high', 'low', 'close', 'rsi_14', 'ma_9',
       'sma_9', 'wma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema',
       'roc'],
      dtype='object')

In [3]:
inputs = data[list(data.columns[6:])]
target = data.iloc[:, [5]]

In [4]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1, shuffle=False)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
X_train_scaled.shape

(12977, 12)

In [7]:
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
y_train_reshaped = y_train.values.reshape(y_train.shape[0], y_train.shape[1], 1)

In [8]:
X_train_reshaped.shape

(12977, 12, 1)

In [9]:
y_train_reshaped.shape

(12977, 1, 1)

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2

2024-03-20 08:02:55.848903: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 08:02:56.584292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 08:02:56.584376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 08:02:56.709760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 08:02:56.958215: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 08:03:01.237333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [11]:
# Define LSTM model
model = Sequential([
    LSTM(50, input_shape=(12, 1), activation='tanh', kernel_regularizer=l2(0.01)),
    #Dense(64, activation='softplus'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train_reshaped, epochs=100, batch_size=32)

Epoch 1/100
406/406 [==============================] - 7s 10ms/step - loss: 8057250.0000
Epoch 2/100
406/406 [==============================] - 4s 10ms/step - loss: 7936039.5000
Epoch 3/100
406/406 [==============================] - 4s 11ms/step - loss: 7824333.0000
Epoch 4/100
406/406 [==============================] - 4s 9ms/step - loss: 7715878.0000
Epoch 5/100
406/406 [==============================] - 4s 10ms/step - loss: 7609412.5000
Epoch 6/100
406/406 [==============================] - 4s 10ms/step - loss: 7502226.5000
Epoch 7/100
406/406 [==============================] - 4s 10ms/step - loss: 7399543.0000
Epoch 8/100
406/406 [==============================] - 4s 11ms/step - loss: 7295495.0000
Epoch 9/100
406/406 [==============================] - 4s 10ms/step - loss: 7192035.5000
Epoch 10/100
406/406 [==============================] - 4s 10ms/step - loss: 7090699.0000
Epoch 11/100
406/406 [==============================] - 4s 11ms/step - loss: 6990041.5000
Epoch 12/100
406/406

In [12]:
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [13]:
X_test_reshaped

array([[[0.36764502],
        [0.20925812],
        [0.20925812],
        ...,
        [0.22726951],
        [0.21238729],
        [0.4816376 ]],

       [[0.46524768],
        [0.20864155],
        [0.20864155],
        ...,
        [0.22657958],
        [0.21144465],
        [0.49135174]],

       [[0.58494364],
        [0.20877558],
        [0.20877558],
        ...,
        [0.22591725],
        [0.21259629],
        [0.51531157]],

       ...,

       [[0.69281312],
        [0.17801109],
        [0.17801109],
        ...,
        [0.18139782],
        [0.18584725],
        [0.5526739 ]],

       [[0.74477209],
        [0.17889544],
        [0.17889544],
        ...,
        [0.18177532],
        [0.18779537],
        [0.55511691]],

       [[0.75898358],
        [0.17998013],
        [0.17998013],
        ...,
        [0.18214526],
        [0.1894418 ],
        [0.55173142]]])

In [14]:
y_pred = model.predict(X_test_reshaped)

102/102 [==============================] - 1s 4ms/step


In [15]:
y_test

,close
13017,1744.47
13018,1765.01
13019,1735.29
13020,1725.50
13021,1725.71
...,...
16257,1638.52
16258,1656.42
16259,1664.93
16260,1667.43


In [16]:
y_pred

array([[1994.5737],
       [1994.5737],
       [1994.5737],
       ...,
       [1994.5737],
       [1994.5737],
       [1994.5737]], dtype=float32)